# Data Cleaning

## Importing libraries and creating a dataframe

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

In [2]:
file_path = Path.home()/'Downloads/1.-Badly-Structured-Sales-Data-1.xlsx'

In [3]:
df = pd.read_excel(file_path)

In [4]:
df

,Segment>>,Consumer,Unnamed: 2,Unnamed: 3,Unnamed: 4,Consumer Total,Corporate,Unnamed: 7,Unnamed: 8,Unnamed: 9,Corporate Total,Home Office,Unnamed: 12,Unnamed: 13,Unnamed: 14,Home Office Total
0,Ship Mode>>,First Class,Same Day,Second Class,Standard Class,NaN,First Class,Same Day,Second Class,Standard Class,NaN,First Class,Same Day,Second Class,Standard Class,NaN
1,Order ID,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CA-2011-100293,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,91.056,91.0560
3,CA-2011-100706,NaN,NaN,129.44,NaN,129.440,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CA-2011-100895,NaN,NaN,NaN,605.47,605.470,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
820,US-2014-166611,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68.742,68.7420,NaN,NaN,NaN,NaN,NaN
821,US-2014-167920,NaN,NaN,1827.51,NaN,1827.510,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
822,US-2014-168116,NaN,NaN,NaN,NaN,NaN,NaN,8167.42,NaN,NaN,8167.4200,NaN,NaN,NaN,NaN,NaN
823,US-2014-168690,NaN,NaN,NaN,2.808,2.808,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.columns

Index(['Segment>>', 'Consumer', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Consumer Total', 'Corporate', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Corporate Total', 'Home Office', 'Unnamed: 12', 'Unnamed: 13',
       'Unnamed: 14', 'Home Office Total'],
      dtype='object')

## Defining functions needed in the clean up process

In [6]:
# This function accepts a list of items(boolean values) and a dataframe
# returns the (column name) if item results to a False value else returns (no value)
def shipmode(items, df):
    for item in items:
        if item is False:
            items.index(item)
            return df.columns[items.index(item)]
        continue
    return np.nan

In [7]:
# This function accepts an item(boolean value) and a string
# returns string if item results to a False else returns (no value)
def segment_(item, text):
    if item is False:
        return text
    return np.nan

## Creating clean dataframes
1. Explore the unclean dataframe(df)
2. Determine the structure of clean data you need
3. Create a plan of Action

In [8]:
# creating a list of dataframes from the original dataframe df
df_consumer = df.loc[:823,['Segment>>', 'Consumer', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4']]
df_corporate = df.loc[:823,['Segment>>', 'Corporate', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9']]
df_home_office = df.loc[:823,['Segment>>', 'Home Office', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14']]

df_groups = [df_consumer, df_corporate, df_home_office]

# Iterating over every new dataframe
for df_group in df_groups:
    msg = df_group.columns[1]
    
    # renaming the columns
    df_group.columns = list(df_group.loc[0])
    df_group.rename(columns={'Ship Mode>>':'Order ID'}, inplace=True)

    # dropping rows that are not needed
    df_group.drop([0,1], inplace=True)

    # setting the index of the new dataframe
    df_group.set_index('Order ID', inplace=True)

    # Adding columns to the new dataframe 1
    """ a column created by iterating over every row, applying a function to a list of values for each iteration
        and appending the return value"""
    ship_mode = []
    for item in df_group.index:
        x = list(df_group.isna().loc[item])
        ship_mode.append(shipmode(x, df_group))
    df_group['Ship mode'] = ship_mode

    """ a column created by iterating over a list of values from a specific column, applying a function to each iteration
        and appending the return value"""
    segment = []
    x = list(df_group['Ship mode'].isna())
    for item in x:
        segment.append(segment_(item, msg))
    df_group['Segment'] = segment

    # Dropping rows from the new dataframe
    df_group.dropna(axis='index', how='all', inplace=True)
    
    # filling every Missing value with zero
    for x in df_group.columns:
        df_group[x].fillna(0, inplace=True)

    df_group['Sales'] = df_group['First Class'] + df_group['Same Day'] + df_group['Second Class'] + df_group['Standard Class']
    
    # Dropping columns from the new dataframe
    df_group.drop(columns=['First Class', 'Same Day', 'Second Class', 'Standard Class'], inplace=True)

## Merging the clean groups of datasets

In [9]:
clean_df = df_consumer.append(df_corporate)

In [10]:
clean_df = clean_df.append(df_home_office)

In [11]:
clean_df.sort_index(inplace=True)

In [12]:
clean_df

,Ship mode,Segment,Sales
Order ID,,,
CA-2011-100293,Standard Class,Home Office,91.056
CA-2011-100706,Second Class,Consumer,129.440
CA-2011-100895,Standard Class,Consumer,605.470
CA-2011-100916,Standard Class,Corporate,788.860
CA-2011-101266,Second Class,Consumer,13.360
...,...,...,...
US-2014-166233,Standard Class,Consumer,24.000
US-2014-166611,Standard Class,Corporate,68.742
US-2014-167920,Second Class,Consumer,1827.510


## Standardizing the data

In [14]:
# reseting the index
clean_df.reset_index(inplace=True)

clean_df['No'] = [num+1 for num, val in enumerate(clean_df['Order ID'])]

clean_df.set_index('No', inplace=True)

In [32]:
clean_df

,Order ID,Ship mode,Segment,Sales
No,,,,
1,CA-2011-100293,Standard Class,Home Office,91.056
2,CA-2011-100706,Second Class,Consumer,129.440
3,CA-2011-100895,Standard Class,Consumer,605.470
4,CA-2011-100916,Standard Class,Corporate,788.860
5,CA-2011-101266,Second Class,Consumer,13.360
...,...,...,...,...
818,US-2014-166233,Standard Class,Consumer,24.000
819,US-2014-166611,Standard Class,Corporate,68.742
820,US-2014-167920,Second Class,Consumer,1827.510


In [33]:
# splitting column
clean_df[['State', 'Year', 'Order Id']] = clean_df['Order ID'].str.split('-', expand=True)

In [34]:
# dropping the column used for splitting
clean_df.drop(columns=['Order ID'], inplace=True)

In [35]:
# rearranging columns based on prefference
clean_df = clean_df[['State', 'Year', 'Order Id', 'Ship mode', 'Segment', 'Sales']]

In [36]:
# validating the data types
clean_df['Year'] = clean_df['Year'].astype(int)

In [37]:
clean_df

,State,Year,Order Id,Ship mode,Segment,Sales
No,,,,,,
1,CA,2011,100293,Standard Class,Home Office,91.056
2,CA,2011,100706,Second Class,Consumer,129.440
3,CA,2011,100895,Standard Class,Consumer,605.470
4,CA,2011,100916,Standard Class,Corporate,788.860
5,CA,2011,101266,Second Class,Consumer,13.360
...,...,...,...,...,...,...
818,US,2014,166233,Standard Class,Consumer,24.000
819,US,2014,166611,Standard Class,Corporate,68.742
820,US,2014,167920,Second Class,Consumer,1827.510
